In [49]:
import numpy as np
import pandas as pd
import sys

# tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

# pymongo
import config
import ppp
import pymongo
from pymongo import MongoClient
import dnspython
import urllib.parse

In [48]:
# Twitter API
access_token = config.access_token
access_token_secret = config.access_token_secret
consumer_key = config.consumer_key
consumer_secret = config.consumer_secret

## SQLITE

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect('test.db')
c = conn.cursor()

In [5]:
c.execute("create table coffee (main_id varchar(40), id varchar(20), name varchar(50), followers integer null, following integer null)")

## JSON to SQLite

In [20]:
import json
with open("data/test-out (2)", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            c.execute("insert into coffee values (?, ?, ?, ?, ?)",
                      [data['id_str'], data['user']['id'], data['user']['screen_name'], data['user']['followers_count'], data['user']['friends_count']])
        except:
            continue

FileNotFoundError: [Errno 2] No such file or directory: 'data/test-out'

In [31]:
cursor = conn.execute("select * from coffee where main_id == '1253318056299749378'")
rows = cursor.fetchall()

In [32]:
for row in rows:
    print(row)

('1253318056299749378', '2168169565', 'jdabc0507s2', 252, 877)


## JSON to MongoDB

In [50]:
# setting mongodb config
username = urllib.parse.quote_plus(ppp.username)
password = urllib.parse.quote_plus(ppp.password)
cluster = MongoClient('mongodb+srv://%s:%s@cluster0-tlu5n.mongodb.net/test?retryWrites=true&w=majority' % (username, password))

In [51]:
# defining cluster and collection
db = cluster["coffee_twitter"]
collection = db["tweets"]

In [11]:
# reading json and pushing tweet contents to MongoDB database
with open("data/test-out (2)", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            idx = data['id_str']
            full_tweet = data['retweeted_status']['extended_tweet']['full_text']
            lang = data['lang']
            retweeted = data['retweeted']
            reply_count = data['reply_count']
            retweet_count = data['retweet_count']
            
            post = {"id": idx, "full_tweet": full_tweet, "lang": lang, "retweeted": retweeted, "reply_count": reply_count,
                    "retweet_count": retweet_count}
            
            collection.insert_one(post)
        except:
            continue

## Find Information

In [12]:
# query
results = collection.find({"id": "1253317430220337152"})
print(results)

In [28]:
for r in results:
    print(r['full_tweet'] + "\n***")

I typically brew my coffee with milk and sugar all at the same time lol. Btw I will drink all of this alone in about an hour or 2 https://t.co/Q9xn6A2qKN
***
I typically brew my coffee with milk and sugar all at the same time lol. Btw I will drink all of this alone in about an hour or 2 https://t.co/Q9xn6A2qKN
***
hey guys, so if you add milk/creamer/sugar/etc to coffee do they count as condiments because you add them to cover to give it flavour? if so, is coffee also a condiment because you add it to water to give it flavour?
***
hey guys, so if you add milk/creamer/sugar/etc to coffee do they count as condiments because you add them to cover to give it flavour? if so, is coffee also a condiment because you add it to water to give it flavour?
***
hey guys, so if you add milk/creamer/sugar/etc to coffee do they count as condiments because you add them to cover to give it flavour? if so, is coffee also a condiment because you add it to water to give it flavour?
***
hey guys, so if you a

In [23]:
# query text
results = collection.find( { "$text": { "$search": "java steam" } } )
print(results)

In [25]:
# query exact phrase
results = collection.find( { "$text": { "$search": "\"coffee sugar\"" } } )
print(results)

In [27]:
# query term exclusion
results = collection.find( { "$text": { "$search": "-black sugar" } } )
print(results)

In [29]:
# query
results = collection.find({"lang": "ja"})
print(results)

In [30]:
for r in results:
    print(r['id'] + "\n***")

1253318056299749378
***
1253320464077750277
***
1253318187745030155
***
1253318393886695424
***
1253321382827487232
***
1253318686896607233
***
1253318726461452291
***
1253319146994950144
***
1253319512163708930
***
1253319701473587209
***
1253319739599773698
***
1253319955166015490
***
1253319959754637314
***
1253322991082991618
***
1253323112134750210
***
1253320464077750277
***
1253323697831571457
***
1253323872020992002
***
1253324361370460167
***
1253324699355889665
***
1253321382827487232
***
1253324961306963969
***
1253325149182410753
***
1253325392275894273
***
1253325567887241222
***
1253326342860374016
***
1253326355346763778
***
1253326360900038657
***
1253326798735020033
***
1253326815579402242
***
1253326835179380736
***
1253326900526632966
***
1253322991082991618
***
1253323112134750210
***
1253327404811997185
***
1253327624463544323
***
1253327636652146689
***
1253327700653043712
***
1253327708731269126
***
1253327724665430023
***
1253323697831571457
***
1253327838272385